In [ ]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [ ]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


## Notebook overview
- Get insight into tokenizer, tokens and doc lengths.
- Test different text truncation thresholds on the baseline.

#### Text truncation -- overview in tokenizer/doc lengths
- tokenize text using tokenizer of mistral, geitje and Llama.
- Check if mistral and geitje indeed have the same tokenizer.
- After getting the tokens, check distribution.
- Truncate text and test multiple thresholds on baseline

Results are saved in txtfiles_tokenizer.pkl, so that txtfiles.pkl is a back-up file, in case anything gets messed up

In [ ]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")

In [ ]:
from transformers import AutoTokenizer

def get_tokens(model_name, df, save_to_path, text_col, new_col_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    all_texts = list(df[text_col].values)

    all_tokens = []
    all_tokens_len = []
    for txt in all_texts:
        tokens = tokenizer.tokenize(txt)
        all_tokens.append(tokens)
        all_tokens_len.append(len(tokens))

    df[new_col_name] = all_tokens
    df[f"count_{new_col_name}"] = all_tokens_len
    df.to_pickle(save_to_path)
    return df

# subdf = df.iloc[0:2]
# # display(subdf)
# get_token_length('Rijgersberg/GEITje-7B-chat-v2', subdf, f"{cf.output_path}/try_out_token_count.pkl", 'text', 'token_count_geitje')

def fraction_token(df, max_token, token_len_col):
    for col in token_len_col:
        print(f"{len(df.loc[df[col]>max_token])} out of {len(df)} ({round(len(df.loc[df[col]>max_token])/len(df)*100, 2)}%) docs exceed a token length of {max_token}")

    for col in token_len_col:
        print(df[col].describe())

    


    

##### Tokenize text

In [ ]:
"""GEITje""" ## not necesarry -> since tokenizer is the same as mistral
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('Rijgersberg/GEITje-7B-chat-v2', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'GEITjeTokens')

"""Mistral"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('mistralai/Mistral-7B-v0.1', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'MistralTokens')

"""Llama"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('meta-llama/Llama-2-7b-hf', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'LlamaTokens')

##### Analyse token length of model tokenizers

In [ ]:
import pandas as pd
tok = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# fraction_token(tok, 4096, ['count_MistralTokens', 'count_LlamaTokens'])

#### Test text truncation on baseline

In [ ]:
import itertools
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC


# load file with baseline function
import sys
sys.path.append('../scripts/') 
import baseline as bf

# load file with truncation function
from truncation import add_truncation_column

# variables for text truncation
DATAFRAME = tok
TEXT_COL = 'text'
TOKENS_COL = 'LlamaTokens'

# variables for baseline
BASELINE_FUNCTION = MultinomialNB()
MODEL_NAME = 'MultinomialNB'
TRAIN_SET = 'train' # must be dev or train
TEST_SET = 'test' # must be val or test
SPLIT_COLUMN = '4split' #must be either 2split or 4split. 2split = data split into train and test. 4split = data split into train, test, dev and val. 
LABEL_COLUMN = 'label'
PREDICTION_PATH = f"{cf.output_path}/predictions/baselineTruncationPredictions.pkl"
OVERVIEW_PATH = f"{cf.output_path}/overview/baselineTruncationOverview.pkl"
# PREDICTION_PATH = f"{cf.output_path}/predictions/tryoutBaselineTruncationPredictions.pkl"
# OVERVIEW_PATH = f"{cf.output_path}/overview/tryoutBaselineTruncationOverview.pkl"
TRUNC_COLUMN = 'trunc_txt'
threshold_combinations =[(100,0), (200,0), (500,0), (1000,0), (2000,0), (100,100),(200,200), (500,500), (1000,1000), (0,100), (0,200), (0,500), (0,1000), (0,2000)]
threshold_combinations =[(200,200), (500,500), (1000,1000), (0,100), (0,200), (0,500), (0,1000), (0,2000)]

# threshold_combinations = [(100,0)]

In [ ]:
# loop through all thresholds and save predictions
for thresholds in threshold_combinations:
    front_threshold = thresholds[0]
    back_threshold = thresholds[1]
    trunc = add_truncation_column(DATAFRAME, TEXT_COL, TOKENS_COL, front_threshold,back_threshold)
    bf.run_baseline(BASELINE_FUNCTION,MODEL_NAME, trunc, SPLIT_COLUMN, TRAIN_SET, TEST_SET, TRUNC_COLUMN, LABEL_COLUMN, PREDICTION_PATH, OVERVIEW_PATH)

In [ ]:
import pandas as pd
PREDICTION_PATH = f"{cf.output_path}/predictions/baselineTruncationPredictions.pkl"
OVERVIEW_PATH = f"{cf.output_path}/overview/baselineTruncationOverview.pkl"

yeet = pd.read_pickle(OVERVIEW_PATH)
yeet = yeet.sort_values(by=['macro_avg_f1', 'accuracy'], ascending=False)
display(yeet)

bl = pd.read_pickle(f"{cf.output_path}/overview/baselineOverview.pkl")


In [ ]:

# ls = yeet.loc[yeet['model']=='LinearSVC']
ls = pd.concat([bl.loc[bl['model']=='LinearSVC'], yeet.loc[yeet['model']=='LinearSVC']]).sort_values(by=['macro_avg_f1', 'accuracy'], ascending=False)
ls = ls.loc[~ls['text_col'].isin(['TruncationLlamaTokensFront2000Back0', 'TruncationLlamaTokensFront1000Back0', 'TruncationLlamaTokensFront1000Back1000', 'TruncationLlamaTokensFront0Back2000',''])]

In [ ]:
display(ls)

In [ ]:
nb = yeet.loc[yeet['model']=='MultinomialNB']
# nb = nb.loc[~nb['text_col'].isin(['TruncationLlamaTokensFront2000Back0', 'TruncationLlamaTokensFront1000Back0', 'TruncationLlamaTokensFront1000Back1000', 'TruncationLlamaTokensFront0Back2000',''])]
display(nb)



### Finding best cut-off point